In [1]:
from transformers import MambaForCausalLM
from torch import nn
import torch

/mnt/ssd-1/gpaulo/miniconda3/envs/new_transformers/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mamba = MambaForCausalLM.from_pretrained("state-spaces/mamba-2.8b-hf")

Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  2.27it/s]


In [3]:
mamba

MambaForCausalLM(
  (backbone): MambaModel(
    (embeddings): Embedding(50280, 2560)
    (layers): ModuleList(
      (0-63): 64 x MambaBlock(
        (norm): MambaRMSNorm()
        (mixer): MambaMixer(
          (conv1d): Conv1d(5120, 5120, kernel_size=(4,), stride=(1,), padding=(3,), groups=5120)
          (act): SiLU()
          (in_proj): Linear(in_features=2560, out_features=10240, bias=False)
          (x_proj): Linear(in_features=5120, out_features=192, bias=False)
          (dt_proj): Linear(in_features=160, out_features=5120, bias=True)
          (out_proj): Linear(in_features=5120, out_features=2560, bias=False)
        )
      )
    )
    (norm_f): MambaRMSNorm()
  )
  (lm_head): Linear(in_features=2560, out_features=50280, bias=False)
)

In [9]:
class UnitarySSM(nn.Module):
    def __init__(self, model: nn.Module):
        super().__init__()
        self.model = model

        # Register a hook for each layer
        self.hooks = []
        self.add_hooks()
        
    def forward(self,x) :
        return self.model(x)

    def remove_hooks(self):
        for h in self.hooks:
            h.remove()

    def add_hooks(self):
        for name, layer in self.model.backbone.layers.named_children():
            print(layer.named_children())
            layer.__name__ = name
            self.hooks.append(layer.register_forward_hook(
                lambda layer, _, output: print(f"{layer.__name__}: output")
            ))


In [10]:
verbose = UnitarySSM(mamba)

<generator object Module.named_children at 0x7f56396dba70>
<generator object Module.named_children at 0x7f56396dba70>
<generator object Module.named_children at 0x7f56396dba70>
<generator object Module.named_children at 0x7f56396dba70>
<generator object Module.named_children at 0x7f56396dba70>
<generator object Module.named_children at 0x7f56396dba70>
<generator object Module.named_children at 0x7f56396dba70>
<generator object Module.named_children at 0x7f56396dba70>
<generator object Module.named_children at 0x7f56396dba70>
<generator object Module.named_children at 0x7f56396dba70>
<generator object Module.named_children at 0x7f56396dba70>
<generator object Module.named_children at 0x7f56396dba70>
<generator object Module.named_children at 0x7f56396dba70>
<generator object Module.named_children at 0x7f56396dba70>
<generator object Module.named_children at 0x7f56396dba70>
<generator object Module.named_children at 0x7f56396dba70>
<generator object Module.named_children at 0x7f56396dba7

In [11]:
output = verbose(torch.tensor([[0]]))

0: output
1: output
2: output
3: output
4: output
5: output
6: output
7: output
8: output
9: output
10: output
11: output
12: output
13: output
14: output
15: output
16: output
17: output
18: output
19: output
20: output
21: output
22: output
23: output
24: output
25: output
26: output
27: output
28: output
29: output
30: output
31: output
32: output
33: output
34: output
35: output
36: output
37: output
38: output
39: output
40: output
41: output
42: output
43: output
44: output
45: output
46: output
47: output
48: output
49: output
50: output
51: output
52: output
53: output
54: output
55: output
56: output
57: output
58: output
59: output
60: output
61: output
62: output
63: output


In [8]:
verbose.remove_hooks()